<h1>Capstone week 3 part 3</h1>
<p>
    Third part of the <b>Segmenting and Clustering Neighborhoods in Toronto</b> -assignment.
    
<h3>
    Install and import required dependencies
</h3>

In [107]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import requests
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

<h3>Load the data from the previous assignments</h3>

In [159]:
# Load the data
toronto_neighbourhoods = pd.read_csv('toronto_geospatial_postal_codes.csv')
# Inspect data
toronto_neighbourhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h3>Visualize the map of Toronto</h3>

In [150]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.6534817, -79.3839347.


In [151]:
# Generate at new folium map from latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

<h3>Setup FourSquare credentials</h3>

In [258]:
# Removed from github for safety
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605'
LIMIT = 100

<h3>Explore Toronto neighbourhoods and venues</h3>

<p>Re-use the venue query function from the course for acquiring venues for a neighbourhood given lat, lon. Disable print to avoid clutter.</p>

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Apply getNearbyVenues for the whole dataframe

In [80]:
toronto_venues = getNearbyVenues(
    names = toronto_neighbourhoods['Neighborhood'], 
    latitudes = toronto_neighbourhoods['Latitude'], 
    longitudes = toronto_neighbourhoods['Longitude'])

<h3>Inspect the acquired data</h3>

In [81]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [82]:
toronto_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2136,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
2137,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,McDonald's,43.630007,-79.518041,Fast Food Restaurant
2138,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2139,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
2140,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


In [83]:
toronto_venues.shape

(2141, 7)

Check some venue counts for neighbourhoods

In [84]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
Willowdale,39,39,39,39,39,39
"Willowdale, Newtonbrook",2,2,2,2,2,2
Woburn,3,3,3,3,3,3


<h3>Analyze venue data</h3>

In [132]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the data by neighbourhood and take mean of the occurrance for future analyzing of the most common type of venues in the area.

In [133]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Generate new dataframe with the most common venues data per neighbourhoods

In [249]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [250]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Chinese Restaurant,Breakfast Spot,Market,Martial Arts School,Massage Studio,Medical Center,Luggage Store,Molecular Gastronomy Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Coffee Shop,Skating Rink,Pub,Sandwich Place,Men's Store,Middle Eastern Restaurant,Mexican Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Grocery Store,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Park,Convenience Store
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Pub


<h3>Cluster neighbourhoods</h3>
<p>Use k-means to cluster the neighbourhoods into five clusters</p>

In [251]:
# Setup number of clusters
kclusters = 4

# Create dataframe for clusters and drop the Neighborhood column
toronto_clusters = toronto_grouped.drop('Neighborhood', 1)

# Cluster the data with k means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clusters)

In [252]:
# Add cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_neighbourhoods.copy
# Merge neighbourhoods and venues to get combined details for viewing
toronto_merged = pd.merge(toronto_neighbourhoods, neighborhoods_venues_sorted, how='right', on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1S,Scarborough,Agincourt,43.794200,-79.262029,2,Lounge,Latin American Restaurant,Chinese Restaurant,Breakfast Spot,Market,Martial Arts School,Massage Studio,Medical Center,Luggage Store,Molecular Gastronomy Restaurant
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0,Pizza Place,Gym,Dance Studio,Coffee Shop,Skating Rink,Pub,Sandwich Place,Men's Store,Middle Eastern Restaurant,Mexican Restaurant
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,2,Bank,Coffee Shop,Grocery Store,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Park,Convenience Store
3,M2K,North York,Bayview Village,43.786947,-79.385975,2,Chinese Restaurant,Café,Bank,Japanese Restaurant,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Yoga Studio
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,2,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Pub


Visually inspect the cluster on map

In [253]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Investigate clustered neighbourhoods</h3>

In [254]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0,Pizza Place,Gym,Dance Studio,Coffee Shop,Skating Rink,Pub,Sandwich Place,Men's Store,Middle Eastern Restaurant,Mexican Restaurant
43,M9L,North York,Humber Summit,43.756303,-79.565963,0,Gym,Pizza Place,Mediterranean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center
74,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Pizza Place,Playground,Yoga Studio,Mediterranean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store
89,M4A,North York,Victoria Village,43.725882,-79.315572,0,Hockey Arena,Coffee Shop,Pizza Place,Portuguese Restaurant,Yoga Studio,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
91,M9P,Etobicoke,Westmount,43.696319,-79.532242,0,Pizza Place,Chinese Restaurant,Playground,Discount Store,Sandwich Place,Middle Eastern Restaurant,Intersection,Coffee Shop,Massage Studio,Medical Center


The neighbourhoods in this cluster seem to be the areas with different activities like gyms, hockey arenas, dance and yoga studios and playgrounds.

In [255]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,1,Park,Pool,Women's Store,Hakka Restaurant,Liquor Store,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center
27,M3K,North York,DownsviewEast CFB Toronto,43.737473,-79.464763,1,Bus Stop,Park,Airport,Electronics Store,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
35,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,1,Park,Sushi Restaurant,Jewelry Store,Trail,Mediterranean Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center
49,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1,Park,Sandwich Place,Mediterranean Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center,Performing Arts Venue
51,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Mediterranean Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center,Mobile Phone Shop
55,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,1,Park,Arts & Crafts Store,Playground,Intersection,Men's Store,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant
59,M6L,North York,"North Park, Maple Leaf Park, Upwood Park",43.713756,-79.490074,1,Park,Bakery,Construction & Landscaping,Basketball Court,Yoga Studio,Mediterranean Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station
66,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Fast Food Restaurant,Food & Drink Shop,Medical Center,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Mediterranean Restaurant,Massage Studio
69,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Mediterranean Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center
83,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,1,Park,Convenience Store,Intersection,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Mediterranean Restaurant,Modern European Restaurant


This cluster seems to contain the greener parts of the city with large number of parks and other amenities

In [256]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1S,Scarborough,Agincourt,43.794200,-79.262029,2,Lounge,Latin American Restaurant,Chinese Restaurant,Breakfast Spot,Market,Martial Arts School,Massage Studio,Medical Center,Luggage Store,Molecular Gastronomy Restaurant
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,2,Bank,Coffee Shop,Grocery Store,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Park,Convenience Store
3,M2K,North York,Bayview Village,43.786947,-79.385975,2,Chinese Restaurant,Café,Bank,Japanese Restaurant,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Yoga Studio
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,2,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Pub
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Seafood Restaurant,Farmers Market,Restaurant,Beer Bar,Pharmacy,Museum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,M1R,Scarborough,"Wexford, Maryvale",43.750071,-79.295849,2,Auto Garage,Middle Eastern Restaurant,Bakery,Vietnamese Restaurant,Sandwich Place,Shopping Mall,Men's Store,Miscellaneous Shop,Mexican Restaurant,Metro Station
94,M2N,North York,Willowdale,43.770120,-79.408493,2,Pizza Place,Coffee Shop,Ramen Restaurant,Shopping Mall,Restaurant,Café,Grocery Store,Pet Store,Ice Cream Shop,Fast Food Restaurant
95,M2R,North York,Willowdale,43.782736,-79.442259,2,Pizza Place,Coffee Shop,Ramen Restaurant,Shopping Mall,Restaurant,Café,Grocery Store,Pet Store,Ice Cream Shop,Fast Food Restaurant
97,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Mediterranean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store


These neighbourhoods seem to be more downtown style areas with large concentration of restaurants and coffee shops

In [257]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Garden,Yoga Studio,Mediterranean Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Men's Store,Medical Center,Modern European Restaurant


This neighbourhood seems to stand out from the other neighbourhood with its concentration of gardens and yoga studios.